In [2]:
import nibabel as nib
import numpy as np
import scipy
import torchio as tio
import os
#import nrrd
import pathlib
import shutil
import SimpleITK as sitk
from glob import glob

In [3]:
dr = "C:\\Users\\Farae\\Documents\\GIKIStudies\\Semester7\\FYP\\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\\training\\"

In [4]:
array = os.listdir(dr)

In [5]:
print(dr+array[0]+"\\")

C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\


In [7]:
def random_rotate3D(img_numpy, min_angle, max_angle):
   """
   Returns a random rotated array in the same shape
   :param img_numpy: 3D numpy array
   :param min_angle: in degrees
   :param max_angle: in degrees
   """
   assert img_numpy.ndim == 3, "provide a 3d numpy array"
   assert min_angle < max_angle, "min should be less than max val"
   assert min_angle > -360 or max_angle < 360

   all_axes = [(1, 0), (1, 2), (0, 2)]
   angle = np.random.randint(low=min_angle, high=max_angle+1)
   axes_random_id = np.random.randint(low=0, high=len(all_axes))
   axes = all_axes[axes_random_id]
   return scipy.ndimage.rotate(img_numpy, angle, mode='nearest', axes=axes, reshape=False, prefilter=False)

def fixed_rotate3D(img_numpy, angle):
   """
   Returns a fixed rotated array in the same shape
   :param img_numpy: 3D numpy array
   :param min_angle: in degrees
   """
   assert img_numpy.ndim == 3, "provide a 3d numpy array"
   assert -360 < angle < 360, "angle is out of range"

   all_axes = [(1, 0), (1, 2), (0, 2)]
   axes = all_axes[1]
   return scipy.ndimage.rotate(img_numpy, angle, mode='nearest', axes=axes, reshape=False, prefilter=False)

In [7]:
for i in os.listdir(dr):
  data_dir = dr+i+"\\" 
  t1_files = [file for file in os.listdir(data_dir) if file.lower().startswith('flair') and file.endswith(".nii.gz")]
  t1_files
  for subj_file in t1_files:
      subj_path = os.path.join(data_dir, subj_file)
      newfilename = "strpd_"+subj_file 
      strpd_t1_file = os.path.join(data_dir, newfilename)
      command = f'bse -i "{subj_path}" -o "{strpd_t1_file}"'
      os.system(command)
  stpd_t1_files = [file for file in os.listdir(data_dir) if file.lower().startswith('strpd_') and file.endswith(".nii.gz")]
  stpd_t1_files
  for strpd_subj_file in stpd_t1_files:
      strpd_subj_path = os.path.join(data_dir, strpd_subj_file)
      newnewfilename = "norm_" + strpd_subj_file
      norm_strpd_t1_file = os.path.join(data_dir, newnewfilename)
      command = f'fcm-normalize "{strpd_subj_path}" -o "{norm_strpd_t1_file}"'
      os.system(command)

  norm_t1_files = [file for file in os.listdir(data_dir) if file.lower().startswith('norm_') and file.endswith(".nii.gz")]
  norm_t1_files
  for subj_dir in norm_t1_files:
      norm_strpd_t1_file = os.path.join(data_dir + subj_dir)
      print(norm_strpd_t1_file)
      norm_strpd_t1_img = tio.ScalarImage(norm_strpd_t1_file)
      lbl_file = os.path.join(data_dir + "ground_truth.nii.gz")
      print(lbl_file)
      lbl_img = tio.ScalarImage(lbl_file)

      mni = tio.datasets.Colin27()
      to_mni = tio.Resample(mni.t1.path)
      mni_norm_strpd_t1_img = to_mni(norm_strpd_t1_img)
      mni_lbl_img = to_mni(lbl_img)

      mni_norm_strpd_t1_file = os.path.join(data_dir + "mni_" + subj_dir)
      print(mni_norm_strpd_t1_file)
      mni_norm_strpd_t1_img.save(mni_norm_strpd_t1_file)

      mni_lbl_file = os.path.join(data_dir + "mni_label_" + subj_dir)
      print(mni_lbl_file)
      mni_lbl_img.save(mni_lbl_file)
      print(norm_strpd_t1_img)
      print(lbl_img)
      print(mni_norm_strpd_t1_img)
      print(mni_lbl_img)

  os.listdir(data_dir)
  mni_t1_files = [file for file in os.listdir(data_dir) if file.lower().startswith('mni_norm') and file.endswith(".nii.gz")]
  mni_t1_files
  mni_label_t1_files = [file for file in os.listdir(data_dir) if file.lower().startswith('mni_label') and file.endswith(".nii.gz")]
  mni_label_t1_files
  for subj_dir in mni_t1_files:

    # rotating t1 image
    mni_norm_strpd_t1_file = os.path.join(data_dir, subj_dir)
    print(mni_norm_strpd_t1_file)
    in_img = nib.load(mni_norm_strpd_t1_file)
  #  print(in_img)
    in_img_data = in_img.get_fdata()
  #  print(in_img_data.shape)
  #  print(in_img_data.dtype)  
    out_img_data = fixed_rotate3D(in_img_data, 15)
  #  print(out_img_data.shape)
  #  print(out_img_data.dtype)  
    out_img = nib.Nifti1Image(out_img_data, in_img.affine)
  #  out_img = nib.Nifti1Image(out_img_data, affine=None)
  #  print(out_img)
    out_file = os.path.join(data_dir + "rot_" + subj_dir)
    print(out_file)
    nib.save(out_img, out_file)

C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\norm_strpd_flair_time01_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\ground_truth.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\mni_norm_strpd_flair_time01_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\mni_label_norm_strpd_flair_time01_on_middle_space.nii.gz
ScalarImage(shape: (1, 365, 256, 256); spacing: (0.50, 0.98, 0.98); orientation: RAS+; path: "C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\norm_strpd_flair_time01_on_middle_space.nii.gz")
ScalarImage(shape: (1, 365, 256, 256)

In [8]:
for i in os.listdir(dr):
  data_dir = dr+i+"\\" 
  mni_label_t1_files = [file for file in os.listdir(data_dir) if file.lower().startswith('mni_label') and file.endswith(".nii.gz")]
  for subj_dir in mni_label_t1_files:
  # rotating label image
    mni_lbl_file = os.path.join(data_dir, subj_dir)
    print(mni_lbl_file)
    in_img = nib.load(mni_lbl_file)
  #  print(in_img)
    in_img_data = in_img.get_fdata()
  #  print(in_img_data.shape)
    out_img_data = fixed_rotate3D(in_img_data, 15)
  #  print(out_img_data.shape)
    out_img = nib.Nifti1Image(out_img_data, in_img.affine)
  #  out_img = nib.Nifti1Image(out_img_data, affine=None)
  #  print(out_img)
    out_file = os.path.join(data_dir + "rot_" + subj_dir)
    print(out_file)
    nib.save(out_img, out_file)

C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\mni_label_norm_strpd_flair_time01_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\rot_mni_label_norm_strpd_flair_time01_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\mni_label_norm_strpd_flair_time02_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\013\rot_mni_label_norm_strpd_flair_time02_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\LongitudinalMultipleSclerosisLesionSegmentationChallengeMiccai21\training\015\mni_label_norm_strpd_flair_time01_on_middle_space.nii.gz
C:\Users\Farae\Documents\GIKIStudies\Semester7\FYP\Longitudina